In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
'key2' : ['one', 'two', 'one', 'two', 'one'],
'data1' : np.random.randn(5),
'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-1.779985,-0.619490
1,a,two,1.007342,0.029464
2,b,one,0.879082,-1.414209
3,b,two,0.205829,0.738066
4,a,one,-0.319162,-0.698309


In [3]:
grouped = df.groupby("key1")
grouped["data1"].quantile(.9)

key1
a    0.742041
b    0.811757
Name: data1, dtype: float64

In [6]:
# own agg method to  agg or aggregate
def range_arr(arr):
    return arr.max() - arr.min()
grouped[["data1", "data2"]].agg(range_arr)

,data1,data2
key1,,
a,2.787327,0.727773
b,0.673253,2.152275


In [7]:
# describve also
grouped[["data1", "data2"]].describe()


data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.363935  1.394203 -1.779985 -1.049574 -0.319162  0.344090   
b      2.0  0.542456  0.476062  0.205829  0.374142  0.542456  0.710769   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.007342   3.0 -0.429445  0.399376 -0.698309 -0.658899 -0.619490   
b     0.879082   2.0 -0.338071  1.521889 -1.414209 -0.876140 -0.338071   

                          
           75%       max  
key1                      
a    -0.295013  0.029464  
b     0.199997  0.738066

In [8]:
tips = pd.read_csv("../examples/tips.csv")
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]


,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


In [11]:
# agg func as str
grouped = tips.groupby(["day", "smoker"])
grouped_pct = grouped["tip_pct"]
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [12]:
# mulitple funcs as cols in res
grouped_pct.agg(['mean' , 'std' , range_arr])

mean       std  range_arr
day  smoker                               
Fri  No      0.151650  0.028123   0.067349
     Yes     0.174783  0.051293   0.159925
Sat  No      0.158048  0.039767   0.235193
     Yes     0.147906  0.061375   0.290095
Sun  No      0.160113  0.042347   0.193226
     Yes     0.187250  0.154134   0.644685
Thur No      0.160298  0.038774   0.193350
     Yes     0.163863  0.039389   0.151240

In [13]:
# since lambda func <lambda> name , pass (name, func) args
grouped_pct.agg([('foo', 'mean'), ('range' , range_arr)])

foo     range
day  smoker                    
Fri  No      0.151650  0.067349
     Yes     0.174783  0.159925
Sat  No      0.158048  0.235193
     Yes     0.147906  0.290095
Sun  No      0.160113  0.193226
     Yes     0.187250  0.644685
Thur No      0.160298  0.193350
     Yes     0.163863  0.151240

In [16]:
# Df multiple funcs to multiple cols
funcs = ['count' , 'mean', 'max']
res = grouped[['tip_pct', 'total_bill']].agg(funcs)

res

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [22]:
res['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [24]:
# and (name, func) tuple list
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped[['tip_pct', 'total_bill']].agg(ftuples)

C:\Users\user\AppData\Local\Temp\ipykernel_8416\2977673279.py:3: FutureWarning: The provided callable <function var at 0x000001344DEB0900> is currently using SeriesGroupBy.var. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'var' instead.
  grouped[['tip_pct', 'total_bill']].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [26]:
# diff cols diff func
# mapping to agg
grouped.agg({ 'tip_pct' : ['mean', 'max'] , 'total_bill' : np.var })

C:\Users\user\AppData\Local\Temp\ipykernel_8416\75532124.py:3: FutureWarning: The provided callable <function var at 0x000001344DEB0900> is currently using SeriesGroupBy.var. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'var' instead.
  grouped.agg({ 'tip_pct' : ['mean', 'max'] , 'total_bill' : np.var })


tip_pct            total_bill
                 mean       max         var
day  smoker                                
Fri  No      0.151650  0.187735   25.596333
     Yes     0.174783  0.263480   82.562438
Sat  No      0.158048  0.291990   79.908965
     Yes     0.147906  0.325733  101.387535
Sun  No      0.160113  0.252672   66.099980
     Yes     0.187250  0.710345  109.046044
Thur No      0.160298  0.266312   59.625081
     Yes     0.163863  0.241255   69.808518

In [28]:
# Disable row indexes (mainly heirarchial) with as_index=False or reset_index
tips.groupby(['day', 'smoker'], as_index=False)[['total_bill', 'tip' ,'size']].mean()

,day,smoker,total_bill,tip,size
0,Fri,No,18.420000,2.812500,2.250000
1,Fri,Yes,16.813333,2.714000,2.066667
2,Sat,No,19.661778,3.102889,2.555556
3,Sat,Yes,21.276667,2.875476,2.476190
4,Sun,No,20.506667,3.167895,2.929825
5,Sun,Yes,24.120000,3.516842,2.578947
6,Thur,No,17.113111,2.673778,2.488889
7,Thur,Yes,19.190588,3.030000,2.352941
